# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

# Data exploration

In [2]:
census = pd.read_csv("../data/census_zipcode_level.csv");

In [3]:
census.head()

,Zip,State,Population,White,Black,Native,Asian,Islander,Other,Two,...,Families,Married_couple_families,Married_couple_child_under_18,Married_couple_no_child_under_18,Single_parent_families,Single_parent_child_under_18,Single_parent_no_child_under_18,Non_families,Househoulder_living_alone,Householder_living_with_unrelated
0,35004,Alabama,10418,8495,1657,0,198,0,0,42,...,3031,2265,820,1445,766,351,415,1194,1090,104
1,35005,Alabama,7708,4432,3130,2,0,2,0,26,...,2103,1540,530,1010,563,229,334,824,707,117
2,35006,Alabama,3099,2849,205,9,0,0,0,15,...,857,675,261,414,182,73,109,332,314,18
3,35007,Alabama,26630,19673,3210,0,251,10,28,416,...,6680,5457,2533,2924,1223,604,619,2055,1802,253
4,35010,Alabama,20826,12247,7048,93,203,0,0,327,...,5605,3586,1088,2498,2019,1121,898,2302,2112,190


In [4]:
census.columns

Index(['Zip', 'State', 'Population', 'White', 'Black', 'Native', 'Asian',
       'Islander', 'Other', 'Two', 'Hispanic', 'Female_0to9_Years',
       'Female_10to19_Years', 'Female_20to29_Years', 'Female_30to39_Years',
       'Female_40to49_Years', 'Female_50to59_Years', 'Female_60to69_Years',
       'Female_70p_Years', 'Male_0to9_Years', 'Male_10to19_Years',
       'Male_20to29_Years', 'Male_30to39_Years', 'Male_40to49_Years',
       'Male_50to59_Years', 'Male_60to69_Years', 'Male_70p_Years',
       'Median_household_inc', 'Households', 'Less_than_24999',
       'From_25000_to_49999', 'From_50000_to_74999', 'From_75000_to_99999',
       'From_100000_to_149999', 'From_150000_to_199999', 'From_200000_or_more',
       'Population_25_Years_and_Over', 'No_Diploma_pct', 'High_School_pct',
       'Some_College_pct', 'Bachelors_Degree_pct', 'Graduate_Degree_pct',
       'Family_Households', 'Family_Poverty_pct', 'labor_over_16_years',
       'Unemployment_Rate_pct', 'Housing_Units', 'Household

Looks like we have:
* population density
* race 
* gender
* gender x age
* household income
* education
* family poverty
* workforce participation
* unemployment
* family types

In [5]:
# check data types
census.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32989 entries, 0 to 32988
Data columns (total 58 columns):
Zip                                  32989 non-null int64
State                                32989 non-null object
Population                           32989 non-null int64
White                                32989 non-null int64
Black                                32989 non-null int64
Native                               32989 non-null int64
Asian                                32989 non-null int64
Islander                             32989 non-null int64
Other                                32989 non-null int64
Two                                  32989 non-null int64
Hispanic                             32989 non-null int64
Female_0to9_Years                    32989 non-null int64
Female_10to19_Years                  32989 non-null int64
Female_20to29_Years                  32989 non-null int64
Female_30to39_Years                  32989 non-null int64
Female_40to49_Years 

In [6]:
# change data types
cols_to_change = ['No_Diploma_pct','High_School_pct','Some_College_pct','Bachelors_Degree_pct',
                  'Graduate_Degree_pct','Family_Poverty_pct','Unemployment_Rate_pct']
for col in cols_to_change:
    census.loc[census[col] == ' -   '] = np.NAN
    census[col] = census[col].astype('float')

In [7]:
# check to make sure it's all good now
census.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32989 entries, 0 to 32988
Data columns (total 58 columns):
Zip                                  26733 non-null float64
State                                26733 non-null object
Population                           26733 non-null float64
White                                26733 non-null float64
Black                                26733 non-null float64
Native                               26733 non-null float64
Asian                                26733 non-null float64
Islander                             26733 non-null float64
Other                                26733 non-null float64
Two                                  26733 non-null float64
Hispanic                             26733 non-null float64
Female_0to9_Years                    26733 non-null float64
Female_10to19_Years                  26733 non-null float64
Female_20to29_Years                  26733 non-null float64
Female_30to39_Years                  26733 non-null 

# Aggregate by 3-digit zip code

In [8]:
census['zip_str'] = census.Zip.astype(str)
census['zip_3'] = census.zip_str.str.slice(0, 3)

In [9]:
print('Number of unique 3 digit zip codes: ', len(set(census.zip_3)), 
      '\nNumber of unqiue 5 digit zip codes: ', len(set(census.Zip)), sep='')

Number of unique 3 digit zip codes: 841
Number of unqiue 5 digit zip codes: 32989


`census.zip_3` is now the column to aggregate on.

Need to handle percent and aggregate columns differently. I'll just gather the percent columns here and later I'll aggregate them differently:
* sum columns: variable sum/total population
* percent columns: variable sum/number of 5-digit zip codes in each 3-digit zip code

In [10]:
pct_cols = []
print('Columns that are percents:')
for col in census.columns:
    if col[-3:] == 'pct':
        print(col)
        pct_cols.append(col)

Columns that are percents:
No_Diploma_pct
High_School_pct
Some_College_pct
Bachelors_Degree_pct
Graduate_Degree_pct
Family_Poverty_pct
Unemployment_Rate_pct


In [29]:
census_zip3_aggregated = census.groupby(['zip_3'])[census.columns ^ ['State','Zip','zip_str']].agg(['sum','count'])

In [30]:
census_zip3_aggregated_new = pd.DataFrame()

# calculate %s by 3-digit zip
for col in census.columns ^ ['zip_3','Population','State','Zip','zip_str','zip_3']:
    census_zip3_aggregated_new[col] = census_zip3_aggregated[col]['sum']/census_zip3_aggregated['Population']['sum']

# adjust percent columns
for col in pct_cols:
    census_zip3_aggregated_new[col] = census_zip3_aggregated[col]['sum']/(100*census_zip3_aggregated['Population']['count'])
    

In [31]:
# define a measure for % nonwhite
census_zip3_aggregated_new['nonwhite_pct'] = 1 - census_zip3_aggregated_new['White']

In [35]:
# quick peak
census_zip3_aggregated_new.sample(5)

,Asian,Bachelors_Degree_pct,Black,Families,Family_Households,Family_Poverty_pct,Female_0to9_Years,Female_10to19_Years,Female_20to29_Years,Female_30to39_Years,...,Population_25_Years_and_Over,Single_parent_child_under_18,Single_parent_families,Single_parent_no_child_under_18,Some_College_pct,Two,Unemployment_Rate_pct,White,labor_over_16_years,nonwhite_pct
zip_3,,,,,,,,,,,,,,,,,,,,,
466,0.013252,0.147929,0.196347,0.238083,0.238083,0.191500,0.067584,0.068186,0.066107,0.065345,...,0.656029,0.046908,0.080061,0.033153,0.297143,0.029643,0.101929,0.630134,0.488914,0.369866
493,0.010904,0.134628,0.015852,0.262709,0.262709,0.098047,0.061694,0.073454,0.059175,0.056766,...,0.659911,0.026882,0.046709,0.019827,0.329744,0.017266,0.078372,0.918795,0.499038,0.081205
644,0.012502,0.143066,0.020755,0.260799,0.260799,0.089885,0.051588,0.070509,0.058529,0.050851,...,0.671767,0.026630,0.043013,0.016383,0.274066,0.014696,0.050918,0.929483,0.492031,0.070517
561,0.032708,0.126695,0.014365,0.270046,0.270046,0.072746,0.069999,0.062917,0.049527,0.053976,...,0.679956,0.032260,0.049079,0.016819,0.350475,0.010869,0.037119,0.841961,0.513693,0.158039
811,0.023251,0.160750,0.070795,0.249864,0.249864,0.173917,0.058077,0.065679,0.060108,0.054825,...,0.675967,0.030309,0.062349,0.032040,0.317292,0.010699,0.116792,0.501959,0.472904,0.498041


# Save to new csv

In [36]:
census_zip3_aggregated_new.to_csv("../data/census_zip3.csv", index=True)